# Obtain Categorical Variables for Catchment Areas and Electoral Wards (R)

Output of Catchments and Wards csvs to be merged in merge.ipynb

In [ ]:
# Import libraries
library(sf)
library(data.table)
library(ggplot2)
library(tmap)
library(tidygeocoder)
library(leaflet)
library(dplyr)
library(osmdata)
library(tmaptools)
library(tidyr)
library(readr)

In [2]:
## Load priceper postcodes
priceper = read_csv("schools_geo/Brighton_and_Hove_link_13082022.csv")
# Obtain unique postcodes
unique_postcodes <- unique(priceper$postcode)
postcode_dir <- data.frame(postcode = unique_postcodes)

# Check
nrow(postcode_dir)

## left join postcode_df to unique_postcodes 
brighton_postcodes <- left_join(postcode_dir, postcode_df, by=c("postcode" = "pcds"))
nrow(brighton_postcodes)
brighton_houses <- brighton_postcodes%>%
  #remove NAs or 0
  filter(long != 0 )%>%
  filter(lat != 0)%>%
  st_as_sf(., coords = c("long","lat"), 
                   crs = 4326)

Rows: 114152 Columns: 16
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr  (6): propertytype, duration, postcode, lad21cd, transactionid, CONSTRUC...
dbl  (9): priceper, year, price, id, tfarea, numberrooms, classt, CURRENT_EN...
date (1): dateoftransfer

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 3902

Rows: 35547 Columns: 53
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (42): pcd, pcd2, pcds, oscty, ced, oslaua, osward, parish, osnrth1m, osh...
dbl (10): dointr, doterm, usertype, oseast1m, osgrdind, streg, ur01ind, lat,...
lgl  (1): lep2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


pcds,lat,long
<chr>,<dbl>,<dbl>
BN1 1AA,50.82633,-0.140818
BN1 1AB,50.82296,-0.141790
BN1 1AD,50.82169,-0.142167
BN1 1AE,50.82138,-0.141753
BN1 1AF,50.82248,-0.141426
BN1 1AG,50.82292,-0.142871


[1] 3902

In [3]:
brighton_houses <- st_transform(brighton_houses, crs = 27700)

In [12]:
## Read in geojson file of catchment areas
catchment_areas <- "schools_geo/shapefiles/BrightonSecondaryCatchments.geojson"
catchment_areas <- st_read(catchment_areas)
catchment_areas_sf <- st_as_sf(catchment_areas)

Reading layer `BrightonSecondaryCatchments' from data source 
  `/mnt/c/Users/josia/Documents/code/CASA/diss/3A. data/schools/schools_geo/shapefiles/BrightonSecondaryCatchments.geojson' 
  using driver `GeoJSON'
Simple feature collection with 6 features and 2 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 523887.8 ymin: 100896.4 xmax: 539613.4 ymax: 110190.1
Projected CRS: OSGB36 / British National Grid


In [4]:
## Read in geojson file of electoral wards
wards <- "schools_geo/shapefiles/Wards_December_2022_Boundaries_GB_BFC_1176271257858185555.geojson"
wards <- st_read(wards)
wards_sf <- st_as_sf(wards)

Reading layer `Wards_December_2022_Boundaries_GB_BFC_1176271257858185555' from data source `/mnt/c/Users/josia/Documents/code/CASA/diss/3A. data/schools/schools_geo/shapefiles/Wards_December_2022_Boundaries_GB_BFC_1176271257858185555.geojson' 
  using driver `GeoJSON'
Simple feature collection with 55 features and 12 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 515159.5 ymin: 99378.75 xmax: 549824.7 ymax: 130673
Projected CRS: OSGB36 / British National Grid


In [15]:
# Perform a spatial join based on the spatial relationships (i.e., matching postcodes)
joined_sf <- st_join(brighton_houses, catchment_areas_sf, join = st_within)
head(joined_sf)

,postcode,id,AreaName,geometry
,<chr>,<int>,<chr>,<POINT [m]>
1,BN1 1AD,2,StringerVarndean,POINT (530956 104106)
2,BN1 1AJ,2,StringerVarndean,POINT (530929 104078.6)
3,BN1 1AL,2,StringerVarndean,POINT (530901 104068)
4,BN1 1AN,2,StringerVarndean,POINT (530861 104147.6)
5,BN1 1AP,2,StringerVarndean,POINT (530850 104004.7)
6,BN1 1BG,2,StringerVarndean,POINT (530959 104175)


In [5]:
# Perform a spatial join based on the spatial relationships (i.e., matching postcodes)
joined_sf2 <- st_join(brighton_houses, wards_sf, join = st_within)
head(joined_sf2)

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



,postcode,WD22CD,WD22NM,WD22NMW,LAD22CD,LAD22NM,BNG_E,BNG_N,LONG,LAT,Shape_Leng,OBJECTID,GlobalID,geometry
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<POINT [m]>
1,BN1 1AD,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953,POINT (530956 104106)
2,BN1 1AJ,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953,POINT (530929 104078.6)
3,BN1 1AL,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953,POINT (530901 104068)
4,BN1 1AN,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953,POINT (530861 104147.6)
5,BN1 1AP,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953,POINT (530850 104004.7)
6,BN1 1BG,E05002432,Regency,,E06000043,Brighton and Hove,530633,104382,-0.14665,50.8242,4786.541,790,0b3bd8c8-19cb-4b70-852e-3c4da241a953,POINT (530959 104175)


In [19]:
### Export
# Drop geometry
attributes_only <- st_drop_geometry(joined_sf)
# Drop id column
attributes_only$id <- NULL

# Export the data as a CSV file
write.csv(attributes_only, file = "data/BrightonSecondaryCatchments.csv", row.names = FALSE)

In [6]:
### Export
# Drop geometry
attributes_only2 <- st_drop_geometry(joined_sf2)
# Drop id column
attributes_only2$id <- NULL

# Export the data as a CSV file
write.csv(attributes_only2, file = "data/BrightonElectoralWards.csv", row.names = FALSE)

In [7]:
nrow(attributes_only2)

[1] 3902